In [1]:
import sys
import os
# Add the parent directory to the PYTHONPATH
sys.path.append(os.path.abspath('../'))

import numpy as np
import helper as hp
import pandas as pd
import multiprocessing as mp
from configparser import ConfigParser
from evostrat.init_mlp import MLP
from evostrat.evolution_strategy import EvolutionStrategy
from kinetics.jacobian_solver import check_jacobian


2024-03-07 14:59:13.957271: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 14:59:14.012316: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 14:59:14.013869: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 14:59:14.812792: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#Parse arguments from configfile
configs = ConfigParser()
configs.read('configfile.ini')

n_samples = int(configs['MLP']['n_samples'])

lnminkm = float(configs['CONSTRAINTS']['min_km'])
lnmaxkm = float(configs['CONSTRAINTS']['max_km'])

repeats = int(configs['EVOSTRAT']['repeats'])
save_step = int(configs['EVOSTRAT']['save_step'])
generations = int(configs['EVOSTRAT']['generations'])
pop_size = int(configs['EVOSTRAT']['pop_size'])
noise = float(configs['EVOSTRAT']['noise'])
lr = float(configs['EVOSTRAT']['lr'])
decay = float(configs['EVOSTRAT']['decay'])
ss_idx = int(configs['EVOSTRAT']['ss_idx'])
n_threads = int(configs['EVOSTRAT']['n_threads'])

base = configs['PATHS']['base']
output_path = configs['PATHS']['output_path']
met_model = configs['PATHS']['met_model']
model_file = configs['PATHS']['model_file']
thermo_experiment_file = configs['PATHS']['thermo_experiment_file']
regulation_file = configs['PATHS']['regulation_file']
kinetic_params_file = configs['PATHS']['kinetic_params_file']
steady_states_file = configs['PATHS']['steady_states_file']

reward_flag = int(configs['REWARDS']['reward_flag'])
eig_partition = float(configs['REWARDS']['eig_partition'])
n_consider = int(configs['REWARDS']['n_consider'])

pf_flag = int(configs['PARAMETER_FIXING']['pf_flag'])


In [3]:
path_to_kmodel = f'{base}/{met_model}/kinetic/{model_file}'
path_to_tmodel = f'{base}/{met_model}/thermo/{thermo_experiment_file}'
path_to_parameters = f'{base}/{met_model}/parameters/{kinetic_params_file}'
path_to_regulation = f'{base}/{met_model}/{regulation_file}'
path_to_samples = f'{base}/{met_model}/steady_state_samples/{steady_states_file}'

In [4]:
## Load kinetic and thermodynamic data with regulation  
regulation = pd.read_csv(path_to_regulation, header=0)
# Call solvers from SKimPy
chk_jcbn = check_jacobian()
# Integrate regulation data
chk_jcbn.load_regulation_models(path_to_kmodel, path_to_tmodel, path_to_samples, regulation, 1712) 

2024-03-07 14:59:18,112 - thermomodel_new - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2024-03-07 14:59:20,122 - Unnamed - WARNING - Non integer stoichiometries found ['CYTBO3_4pp', 'LMPD_biomass_c_1_420', 'CYTBDpp'] change to integer for linear dependencies
2024-03-07 14:59:21,450 - Unnamed - WARNING - Non integer stoichiometries found ['LMPD_biomass_c_1_420', 'CYTBDpp', 'CYTBO3_4pp'] change to integer for linear dependencies


In [5]:
# Load steady states data into the model
steady_states = pd.read_csv(path_to_samples, header=0, index_col=0).iloc[ss_idx, 0:]
chk_jcbn._load_ssprofile(steady_states)  ## Integrate steady state information

In [6]:
k_names = []
k_regulation = []

for k, v in chk_jcbn.kmodel.parameters.items():
    if k.startswith("km_") or "activator" in k or "inhibitor" in k or "activation" in k or "inhibition" in k:
        if k not in k_names:
            k_names.append(k)
            
        if "activator" in k or "inhibitor" in k or "activation" in k or "inhibition" in k:
            k_regulation.append(k)

print(f"The number of k for generation is {len(k_names)}, \nand the number of regulation parameters is {len(k_regulation)}.")

The number of k for generation is 464, 
and the number of regulation parameters is 106.


In [7]:
parameter_sample= {v.symbol: v.value for k,v in chk_jcbn.kmodel.parameters.items()}

print(parameter_sample)

{vmax_forward_ACALDtex: None, k_equilibrium_ACALDtex: 25.82898854650926, acald_e: None, vmax_forward_ACALDtpp: None, kcat_forward_ACALDtpp: None, k_equilibrium_ACALDtpp: 7.768867659479719, km_substrate_ACALDtpp: None, km_product_ACALDtpp: None, vmax_backward_ACALDtpp: None, total_enzyme_concentration_ACALDtpp: None, k1_fwd_ACALDtpp: None, k1_bwd_ACALDtpp: None, k2_fwd_ACALDtpp: None, k2_bwd_ACALDtpp: None, vmax_forward_ACONTa: None, kcat_forward_ACONTa: None, k_equilibrium_ACONTa: 0.05060241976744095, km_substrate_ACONTa: None, km_product_ACONTa: None, vmax_backward_ACONTa: None, total_enzyme_concentration_ACONTa: None, k1_fwd_ACONTa: None, k1_bwd_ACONTa: None, k2_fwd_ACONTa: None, k2_bwd_ACONTa: None, vmax_forward_ACONTb: None, kcat_forward_ACONTb: None, k_equilibrium_ACONTb: 0.31367346932397744, km_substrate_ACONTb: None, km_product_ACONTb: None, vmax_backward_ACONTb: None, total_enzyme_concentration_ACONTb: None, k1_fwd_ACONTb: None, k1_bwd_ACONTb: None, k2_fwd_ACONTb: None, k2_bwd_

In [8]:
# Create a DataFrame from list1
df = pd.DataFrame(chk_jcbn.kmodel.parameters.keys(), columns=['Full List'])

# Initialize the second column with NaNs or an appropriate placeholder
df['Selected'] = pd.Series([None] * len(df))

# Populate the 'Selected' column with values from list2 where they match values in 'Full List'
for item in k_names:
    if item in df['Full List'].values:
        df.loc[df['Full List'] == item, 'Selected'] = item

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('matched_list.csv', index=False)

                   Full List Selected
0      vmax_forward_ACALDtex     None
1     k_equilibrium_ACALDtex     None
2                    acald_e     None
3      vmax_forward_ACALDtpp     None
4      kcat_forward_ACALDtpp     None
...                      ...      ...
1037           cell_volume_c     None
1038                volume_e     None
1039           cell_volume_e     None
1040                volume_p     None
1041           cell_volume_p     None

[1042 rows x 2 columns]


In [9]:
# declare reward functions
def reward_func(weights):
    """
    evaluate reward for a set of generated kinetic parameter sets
    :param gen_params: agent generated kinetic parameter sets
    :return: reward
    """

    global calc_eig
    global n_samples
    global eig_partition
    global n_threads
    global reward_flag
    global gen_params
    global n_consider

    def calc_eig(gen_param):
        chk_jcbn._prepare_parameters(gen_param, k_names)
        max_eig = chk_jcbn.calc_eigenvalues_recal_vmax()
        return max_eig

    pool = mp.Pool(n_threads)

    mlp.generator.set_weights(weights)
    gen_params = mlp.sample_parameters()
    gen_params = [[params] for params in gen_params]

    # max_eig = pool.map(calc_eig, gen_params)
    for gen_param in gen_params:
        calc_eig(gen_param)

    # return ""
    max_eig = np.array([this_eig for eig in max_eig for this_eig in eig])

    if reward_flag == 0:
        max_neg_eig = np.min(max_eig)
        if max_neg_eig > eig_partition:
            this_reward = 0.01 / (1 + np.exp(max_neg_eig - eig_partition))
        else:
            this_reward = len(np.where(max_eig <= eig_partition)[0]) / n_samples
        pool.close()
    elif reward_flag == 1:
        max_eig.sort()
        considered_avg = sum(max_eig[:n_consider]) / n_consider
        this_reward = np.exp(-0.1 * considered_avg) / 2
        pool.close()
    return this_reward

In [10]:
chk_jcbn.parameter_sample_set

[]

In [11]:

for rep in range(1):
    print(f"repeat {rep}")
    cond_class = 1
    # Call neural network agent
    mlp = MLP(cond_class, lnminkm, lnmaxkm, n_samples, k_names, len(k_names), param_fixing=pf_flag)
    '''
    ## Snippet to start from previously saved weights 
    load_weights = hp.load_pkl('/path/to/weights')
    mlp.generator.set_weights(load_weights)
    '''
    init_dict = mlp.generator.get_weights()

    this_savepath = f'../{output_path}/repeat_{rep}/'
    os.makedirs(this_savepath, exist_ok=True)

    es = EvolutionStrategy(mlp.generator.get_weights(),
                           reward_func, this_savepath,
                           population_size=pop_size,
                           sigma=noise,  # noise std deviation
                           learning_rate=lr,
                           decay=decay,
                           num_threads=1)
    
    rewards = es.run(generations, print_step=save_step)
    hp.save_pkl(f'../{this_savepath}/rewards', rewards)



repeat 0
99
starting


2024-03-07 14:59:54.662042: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


4/4 [==============================] - 0s 5ms/step


ValueError: array must not contain infs or NaNs

In [ ]:
chk_jcbn.conc_series

0     2.569070e+02
1     2.079697e+03
2     1.220329e+04
3     6.569170e+06
4     3.824023e+03
          ...     
85    7.803086e+05
86    1.541987e+04
87    5.000000e+05
88    1.038316e+02
89    1.011325e+02
Length: 90, dtype: float64

In [ ]:
chk_jcbn.parameter_sample_set[0]

{vmax_forward_ACALDtex: 12783813.269378724,
 k_equilibrium_ACALDtex: 25.82898854650926,
 acald_e: None,
 vmax_forward_ACALDtpp: 511844413.1855303,
 kcat_forward_ACALDtpp: None,
 k_equilibrium_ACALDtpp: 7.768867659479719,
 km_substrate_ACALDtpp: 40874.84375,
 km_product_ACALDtpp: 36970.37109375,
 vmax_backward_ACALDtpp: None,
 total_enzyme_concentration_ACALDtpp: None,
 k1_fwd_ACALDtpp: None,
 k1_bwd_ACALDtpp: None,
 k2_fwd_ACALDtpp: None,
 k2_bwd_ACALDtpp: None,
 vmax_forward_ACONTa: 1132065248.9157417,
 kcat_forward_ACONTa: None,
 k_equilibrium_ACONTa: 0.05060241976744095,
 km_substrate_ACONTa: 37637.79296875,
 km_product_ACONTa: 124472.4140625,
 vmax_backward_ACONTa: None,
 total_enzyme_concentration_ACONTa: None,
 k1_fwd_ACONTa: None,
 k1_bwd_ACONTa: None,
 k2_fwd_ACONTa: None,
 k2_bwd_ACONTa: None,
 vmax_forward_ACONTb: 1342363259662.9888,
 kcat_forward_ACONTb: None,
 k_equilibrium_ACONTb: 0.31367346932397744,
 km_substrate_ACONTb: 2613.295654296875,
 km_product_ACONTb: 34216.95703

In [ ]:
print(chk_jcbn.jacobian_set[0])
print("Shape:", chk_jcbn.jacobian_set[20].shape)
print("Non-zero elements:", chk_jcbn.jacobian_set[20].nnz)



  (0, 0)	nan
  (1, 0)	13096.480554057232
  (1, 1)	-47895.85916685859
  (7, 1)	46147.92088533753
  (10, 1)	-46147.92088533753
  (11, 1)	-46208.199556986234
  (22, 1)	46208.199556986234
  (14, 1)	60.278671648699216
  (0, 1)	14646.428444012887
  (7, 2)	46.71220959660032
  (9, 2)	-46.71220959660032
  (68, 2)	-46.71220959660032
  (84, 2)	46.49257888913179
  (4, 2)	98718527.22854733
  (45, 2)	-10540.282985794873
  (46, 2)	-10540.282985794873
  (18, 2)	0.21963070746852606
  (2, 2)	-98711747.56434843
  (3, 2)	3760.6187868974794
  (7, 3)	-346.46921370716063
  (9, 3)	77.5734631367787
  (68, 3)	-2.899079150227628
  (84, 3)	-0.4548552932772048
  (10, 3)	2.1031730638375485
  (82, 3)	-2.1031730638375485
  :	:
  (38, 80)	209.10025883121486
  (58, 80)	209.10025883121486
  (74, 80)	-209.10025883121486
  (81, 81)	nan
  (7, 81)	16551.66329561845
  (8, 81)	-148964.969660566
  (59, 81)	16551.66329561845
  (9, 82)	-3.000534550555328
  (10, 82)	3.000534550555328
  (82, 82)	-7.400747840218347
  (84, 82)	3.000

In [ ]:
len(chk_jcbn.jacobian_set)

1472

In [12]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix

# Creating a sample matrix with some NaN values
matrix = chk_jcbn.jacobian_set[0]
dense_matrix = matrix.toarray()

# Use a colormap that represents NaN values with grey
# 'viridis' is just an example; you can use any suitable colormap
cmap = plt.cm.viridis
cmap.set_bad(color='grey')

# Create the heatmap
plt.figure(figsize=(8, 6))
plt.imshow(dense_matrix, interpolation='nearest', cmap=cmap)

# Add color bar for reference
plt.colorbar()

# Add title and labels as needed
plt.title('Heatmap of Matrix')
plt.xlabel('Column')
plt.ylabel('Row')

# Show the plot
plt.show()

IndexError: list index out of range

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csc_matrix

# Create the output directory if it doesn't exist
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Assuming sparse_matrices is your list of csc_matrix objects
# sparse_matrices = [csc_matrix1, csc_matrix2, ..., csc_matrix100]

for i, sparse_matrix in enumerate(chk_jcbn.jacobian_set):
    # Convert the sparse matrix to a dense array
    dense_matrix = sparse_matrix.toarray()
    
    # Use a colormap and set NaN values to appear grey
    cmap = plt.cm.viridis
    cmap.set_bad(color='grey')
    
    # Create the heatmap
    plt.figure(figsize=(8, 6))
    plt.imshow(dense_matrix, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    plt.title(f'Heatmap {i+1}')
    plt.xlabel('Column')
    plt.ylabel('Row')
    
    # Save the plot to the output directory with a unique name
    plt.savefig(f'{output_dir}/heatmap_{i+1}.png')
    plt.close()  # Close the plot to free up memory

In [ ]:
type(chk_jcbn.jacobian_set[0])

scipy.sparse._csc.csc_matrix

In [ ]:
chk_jcbn.kmodel.jacobian_fun

## Save parameters

In [ ]:
set_0 = gen_params[0][0]
gen_flat = [inner_list[0] for inner_list in gen_params]


In [ ]:
# Convert the numpy array to a DataFrame
df = pd.DataFrame(gen_flat, columns=k_names)

# Save the DataFrame to a CSV file
df.to_csv('gen_parms.csv', index=False)